In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import dataset_utils
import NN

In [143]:
import dataset_utils

In [144]:
DATA_DIR = "pointnet_data"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: standing
processing class: sitting


In [145]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [146]:
print(train_points)

[[[-4.399e-01  2.570e+00  9.727e-01 -2.505e-01  2.240e+02]
  [-5.420e-01  2.561e+00  1.252e+00 -3.757e-01  2.810e+02]
  [-8.057e-01  3.117e+00  8.516e-01  5.010e-01  2.880e+02]
  ...
  [-5.503e-01  2.578e+00  1.606e+00 -5.010e-01  2.600e+02]
  [-9.092e-01  3.297e+00  4.648e-01  5.010e-01  2.340e+02]
  [-7.285e-01  3.258e+00  8.496e-01  5.010e-01  3.400e+02]]

 [[-7.461e-01  3.445e+00  4.014e-01  3.757e-01  2.210e+02]
  [-7.461e-01  3.184e+00  6.299e-01  3.757e-01  2.370e+02]
  [-8.574e-01  3.324e+00  5.352e-01  3.757e-01  2.260e+02]
  ...
  [-7.124e-01  2.531e+00  1.045e+00 -8.765e-01  2.830e+02]
  [-7.061e-01  2.961e+00  6.855e-01  5.010e-01  2.190e+02]
  [-8.320e-01  3.082e+00  8.369e-01  5.010e-01  2.910e+02]]

 [[-5.088e-01  2.898e+00  6.611e-01 -3.757e-01  3.150e+02]
  [-3.567e-01  2.311e+00  6.377e-01 -8.765e-01  2.970e+02]
  [-6.973e-01  3.061e+00  8.848e-01  3.757e-01  3.450e+02]
  ...
  [-5.088e-01  3.215e+00  6.094e-01 -6.260e-01  3.150e+02]
  [-4.727e-01  2.451e+00  1.364e+0

In [147]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [148]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [149]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [150]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [151]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [152]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# 2. LSTM layer
model.add(layers.LSTM(50, return_sequences=True))
model.add(layers.LSTM(50))

# 3. Dense layers
model.add(layers.Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(layers.Dense(1, activation='linear'))  # Output layer (you can adjust the output size and activation)


In [153]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_36 (Conv1D)              │ (None, None, 32)       │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, None, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_37 (Conv1D)              │ (None, None, 64)       │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, None, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_32 (MaxPooling1D) │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_48 (Dropout)            │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_38 (Conv1D)              │ (None, None, 512)      │        33,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, None, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_33 (MaxPooling1D) │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_49 (Dropout)            │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, None, 50)       │       112,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 256)            │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 218,433 (853.25 KB)

 Trainable params: 216,449 (845.50 KB)

 Non-trainable params: 1,984 (7.75 KB)

In [154]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, batch_size=16, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 3.6415 - mae: 1.5050 - val_loss: 0.2797 - val_mae: 0.4951
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 1.9390 - mae: 1.1321 - val_loss: 0.2692 - val_mae: 0.4834
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 1.5690 - mae: 0.9835 - val_loss: 0.2359 - val_mae: 0.4848
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 1.4931 - mae: 0.9336 - val_loss: 0.2295 - val_mae: 0.4738
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.9936 - mae: 0.8006 - val_loss: 0.1780 - val_mae: 0.4012
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.9743 - mae: 0.7811 - val_loss: 0.0880 - val_mae: 0.2419
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.9831 - mae: 0.7658 - val_loss: 0.0649 - val_mae: 0.2103
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.6607 - mae: 0.6460 - val_loss: 0.0779 - val_mae: 0.2481
Epoch 9/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.525

In [155]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0040 - mae: 0.0490  


In [92]:
# pickle.dump(model, open("model.pkl", "wb"))

In [156]:
print(test_acc)

0.0693371519446373


In [ ]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# Flatten the output
model.add(layers.Flatten())

# Summary of the CNN model
model.summary()
